# 데이터 추출

In [ ]:
import re
import csv

def parse_txt_to_csv(input_path, output_path):
    """
    input_path: txt 파일 경로
    output_path: csv 파일 경로
    """

    # txt 전체 내용을 불러오기
    with open(input_path, 'r', encoding='utf-8') as f:
        text = f.read()

    # -------------------------------------------------------
    # 1) Test sample 별로 나누기
    # -------------------------------------------------------
    #  - 예) Test sample #1 ... Test sample #2 ...
    #  - lookahead로 '(?=Test sample #\d+|$)' : 다음 Test sample 시작 or 파일 끝
    test_samples = re.findall(
        r'(?s)Test sample #(\d+)(.*?)(?=Test sample #\d+|$)',
        text
    )
    # test_samples -> [('1', '...'), ('2', '...'), ...]

    rows = []  # CSV에 담을 결과

    # 정규식 패턴 준비
    # 1) "도입부" (skip 대상) 정규식
    #    - You are a helpful assistant.*?When you generate an answer, please start it with the following phrase: 'The final answer is:'and then provide your solution.
    #    - DOTALL 모드로 최소 일치(.*?) 사용
    skip_pattern = re.compile(
        r"(?s)You are a helpful assistant.*?"
        r"When you generate an answer, please start it with the following phrase:\s*'The final answer is:'and then provide your solution\."
    )

    # 2) "The final answer is:" 이후를 잡아내는 정규식
    #    reasoning -> group(1)
    #    final answer + 이후 -> group(2)
    fa_pattern = re.compile(
        r"(?s)(.*?)The final answer is:\s*(.*)"
    )

    for sample_num_str, sample_block in test_samples:
        문제번호 = sample_num_str.strip()

        # -------------------------------------------------------
        # 2) [1], [2], [3], [4], [5] 블록 분리
        # -------------------------------------------------------
        blocks = re.findall(
            r'(?s)\[(\d+)\]\s*([^\n]+)\n(.*?)(?=\[\d+\]|$)',
            sample_block
        )
        # 결과 예시: [ ('1', 'total G-NLL + max:', '...본문...'), ... ]

        for block_number, block_type_line, block_content in blocks:
            # 블록 종류
            block_type = block_type_line.replace(':','').strip()

            # 1) 먼저 "도입부(skip_pattern)" 를 찾아 제거
            skip_match = skip_pattern.search(block_content)
            if not skip_match:
                # "You are a helpful assistant... 'The final answer is:'and then provide your solution."
                # 이 없으면 사용자가 말한 "그때는 그냥 skip" 을 어떻게 해석할지 결정
                # 여기서는 "없으면 블록 전체 스킵"
                continue

            # skip_match.end() 위치부터 끝까지 사용
            content_after_skip = block_content[skip_match.end():]

            # 2) 이제 content_after_skip 에서
            #    "The final answer is:" 를 찾는다.
            #    reasoning, final_answer 로 분리
            fa_match = fa_pattern.search(content_after_skip)
            if not fa_match:
                # "The final answer is:"가 없다면 스킵
                continue

            # reasoning 은 group(1), final answer + 이후는 group(2)
            reasoning_part = fa_match.group(1).strip()
            fa_and_after = fa_match.group(2).strip()

            # 3) Elapsed time 앞까지만 final answer 로 저장
            final_answer_part = fa_and_after
            match_et = re.search(
                r'(?s)(.*?)(?:Elapsed time:.*)',
                fa_and_after
            )
            if match_et:
                final_answer_part = match_et.group(1).strip()

            # 4) elapsed time 추출
            elapsed_time = ""
            match_time = re.search(
                r'Elapsed time:\s*([\d\.]+)\s*seconds',
                block_content
            )
            if match_time:
                elapsed_time = match_time.group(1)

            # 5) CSV로 담을 row 구성
            row = {
                "문제 번호": 문제번호,
                "종류": block_type,
                "final answer is": final_answer_part,
                "reasoning": reasoning_part,
                "소요 시간": elapsed_time
            }
            rows.append(row)

    # ----------------------------------------------
    # CSV로 저장
    # ----------------------------------------------
    fieldnames = ["문제 번호", "종류", "final answer is", "reasoning", "소요 시간"]
    with open(output_path, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(rows)

    print(f"Done! Parsed results are saved in {output_path}")



In [ ]:
if __name__ == "__main__":
    input_txt_file = 'txt파일 경로.txt'

    output_csv_file = "parsed_results.csv"  # 저장할 csv 파일 경로

    parse_txt_to_csv(input_txt_file, output_csv_file)

# without CoT Prompt

In [ ]:
import re
import csv

def parse_txt_to_csv(input_path, output_path):
    """
    주어진 txt 파일을 파싱하여 CSV로 저장합니다.

    input_path: txt 파일 경로
    output_path: csv 파일 경로
    """

    # txt 전체 내용을 불러오기
    with open(input_path, 'r', encoding='utf-8') as f:
        text = f.read()

    # -------------------------------------------------------
    # 1) Test sample 별로 나누기
    # -------------------------------------------------------
    test_samples = re.findall(
        r'(?s)Test sample #(\d+)(.*?)(?=Test sample #\d+|$)',
        text
    )

    rows = []

    for sample_num_str, sample_block in test_samples:
        number_q = sample_num_str.strip()

        # -------------------------------------------------------
        # 2) [1], [2], [3]... 블록 분리
        # -------------------------------------------------------
        #  - [n] 다음 줄에 블록 타입이 나오고, 그 다음 줄부터 내용을 캡처
        blocks = re.findall(
            r'(?s)\[(\d+)\]\s*(.*?)\n(.*?)(?=\[\d+\]|$)',
            sample_block
        )

        for block_number, block_type_line, block_content in blocks:
            # 블록 종류
            block_type = block_type_line.replace(':', '').strip()

            # -------------------------------------------------------
            # 3) Elapsed time 추출
            # -------------------------------------------------------
            match_time = re.search(
                r'Elapsed time:\s*([\d\.]+)\s*seconds',
                block_content
            )
            elapsed_time = ""
            if match_time:
                elapsed_time = match_time.group(1)

            # -------------------------------------------------------
            # 4) answer에 들어갈 내용(기존 reasoning 내용)
            #    -> Elapsed time 구문은 제거해서 저장
            # -------------------------------------------------------
            answer_text = re.sub(
                r'Elapsed time:\s*[\d\.]+\s*seconds',
                '',
                block_content
            ).strip()

            # -------------------------------------------------------
            # CSV로 담을 row 구성
            # -------------------------------------------------------
            row = {
                "number": number_q,
                "type": block_type,
                "answer": answer_text,
                "elapsed_time": elapsed_time
            }
            rows.append(row)

    # ----------------------------------------------
    # 5) CSV로 저장
    # ----------------------------------------------
    fieldnames = ["number", "type", "answer", "elapsed_time"]
    with open(output_path, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(rows)

    print(f"Done! Parsed results are saved in {output_path}")


In [ ]:
if __name__ == "__main__":
    input_txt_file = 'txt파일 경로.txt'

    output_csv_file = "parsed_results.csv"  # 저장할 csv 파일 경로

    parse_txt_to_csv(input_txt_file, output_csv_file)